<div align="center"><h1>HPC / SENAI / Hackathon (2023.2)<br>
Brute Force Algorithm </h1></div>

**Murilo Boratto**$^1$

$^1$ Supercomputing Center SENAI CIMATEC, Salvador, Bahia, Brazil 

## Introdução

As técnicas de paralelismo compreendem em aplicação de estratégias baseadas na utilização de processamento paralelo manipulando diferentes recursos computacionais. Alguma dessas técnicas compreendem a utilização de bibliotecas paralelas como `OpenMP`, `MPI` e `CUDA`. Cada uma dessas bibliotecas consiste na manipulação de recursos computacionais diferentes, podendo ser utilizadas de forma híbrida a fim da obtenção de máximo desempenho. No OpenMP e CUDA manipulamos *Threads*, enquanto no MPI *Processos*, sendo de real relevância os impactos destas unidades de processamento frente aos recursos computacionais. A seguir será apresentado um código sequencial para a quebra de senha de até 20 caracteres utilizando um algoritmo de *Força Bruta*. O objetivo básico será inserir técnicas de paralelismo ao código, tal que serão considerados alguns itens nos produtos finais: 

* `Análise dos Custos Computacionais das Aplicações Sequênciais e Paralelas`
    + Profilling CPU (gprof)
    + Profiling GPU (nsys)
* `Estudo das Estruturas Algorítmicas das Aplicações Paralelas`
    + Modelos Algorítmicos Aplicados
    + Características da inserção da API
* `Análise de Desempenho`
     + Experimentação de Parâmetros Ótimos (Melhores valores de Processos, Threads e Grid Computacional)
     + Indices de eficiência (Speedup)

## Regras do Hackathon

* Os idiomas oficiais desse HPC Hackathon são: inglês e português;

* Este ano a competição será em grupos de 2 ou 3 pessoas;

* Tópico Principal: Portabilidade e Otimização de Código;
    
* Os participantes disponibilizarão os resultados através um repositório git pessoal que será configurado pelos participantes e/ou pela ferramenta GOOGLE COLAB;

* Além do código modificado, a resolução deve conter scripts de execução automática para obter os Parâmetros Ótimos e os Speedups;

* O código produzido será avaliado em 2 pontos: desempenho e speedup;

* Os participantes devem codificar e comentar seus códigos;

* Os participantes acessarão o supercomputador via ssh com suas contas previamente configuradas;

* As habilidades necessárias são: Git, C, C++, Python, Unix, shell, bash, OpenMP, CUDA, MPI;

Boa sorte e boa codificação!

## Código Sequencial 

O código acima tem como finalidade a simulação de uma quebra de senha, através da estrátegia de *Força Bruta*, ou seja, teste de todas as possbilidades, uma a uma, até encontrar a solução. Faz-se perceber que o número de possibilidades em um alfabeto de 26 letras em uma senha de 20 caracteres é enorme, por isso, ao tentar cada combinação exige um grande poder computacional, reberberando em um alto tempo de execução, especialmente para um código sequencial. <br>  

In [ ]:
%%writefile bruteForce.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>

//97 to 122 use only lowercase letters
//65 to 90 use only capital letters
//48 to 57 use only numbers

#define START_CHAR 48
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

long long my_pow(long long x, int y)
{
  long long res = 1;
  if (y==0)
    return res;
  else
    return x * my_pow(x, y-1);
}

void bruteForce(char *pass) 
{
  char force[MAXIMUM_PASSWORD];
  int pass_b26[MAXIMUM_PASSWORD];
    
  long long int j;
  long long int pass_decimal = 0;
  int base = END_CHAR - START_CHAR + 2;

  int size = strlen(pass);

  for(int i = 0; i < MAXIMUM_PASSWORD; i++)
    force[i] = '\0';

  printf("Try to broke the password: %s\n", pass);

  for(int i = 0; i < size; i++)
    pass_b26[i] = (int) pass[i] - START_CHAR + 1; 

  for(int i = size - 1; i > -1; i--)
    pass_decimal += (long long int) pass_b26[i] * my_pow(base, i);

  long long int max = my_pow(base, size);
  char s[MAXIMUM_PASSWORD];

  for(j = 0; j < max; j++){
    if(j == pass_decimal){
      printf("Found password!\n");
      int index = 0;

      printf("Password in decimal base: %lli\n", j);
      while(j > 0){
        s[index++] = START_CHAR + j%base-1;
        j /= base;
      }
      s[index] = '\0';
      printf("Found password: %s\n", s);
      break;
    }
  }

}

int main(int argc, char **argv) 
{
  char password[MAXIMUM_PASSWORD];
  strcpy(password, argv[1]);
  time_t t1, t2;
  double dif;

  time (&t1);
    bruteForce(password);
  time (&t2);

  dif = difftime (t2, t1);

  printf("\n%1.2f seconds\n", dif);

  return 0;
}

In [ ]:
!gcc bruteForce.c -o bruteForce -std=c99 -O3

In [ ]:
!./bruteForce senhatez

O algoritmo realiza a leitura de uma string qualquer passada como parâmetro pelo terminal, respeitando os limites pré-estabelecidos, e a transforma em sua representação numérica de inteiro. Um vetor de inteiros é criado para armazena-la, letra por letra, transformando os valores de acordo com a posição das letras na tabela ASCII em relação ao caractere inicial de referência. Dessa forma, levando em conta, por exemplo, apenas letras minúsculas, a letra "a" se torna "1", "b" se torna "2" e assim por diante. Logo depois, tratando esse vetor de inteiros como um número em base 26 em ele é "encriptado" em um novo inteiro de base númerica 10, o que pode gerar um valor extremamente alto que cresce exponencialmente a cada letra da palavra. Este processo é representado pela figura a seguir:

<div style="text-align: justify">
<center><img src="images/stringToDecimal.png" width="500" height=auto></center>
<div align="center"><b>Figura 1 -</b> Processos de transformação da senha até sua representação em decimal.</div>
</div>


Só a partir desse ponto a técnica de *Força Bruta* entra em ação para efetivamente decriptar a senha, comparando número a número dentro do laço _for_. Quando a senha é encontrada ela é transformada novamente na string original como representado pela Figura 2, então o processo se encerra, exibindo o tempo total de execução do código.<br>

Códigos que utilizam este método são implementações em potencial para a aplicação de técnicas de paralelização, devido que há uma independêcia entre as tentativas de comparação entre combinações, e que a implementação destas técnicas podem aumentar a eficiência do código de forma exponencial. 

<div style="text-align: justify">
<center><img src="images/decimalToString.png" width="600" height=auto></center>
<div align="center"><b>Figura 2 -</b> Processos de busca e retorno da base 10 para a senha original. </div>
</div>

## Análise dos Custos Computacionais da Aplicação Sequencial (Profilling)

Conhecendo o algoritmo sequencial faz-se necessário mensurar os pontos do código que exigem maior custo computacional da aplicação,  para que possamos focar nossos esforços na paralelização desses trechos, desta maneira, conseguimos trabalhar de forma inteligente, onde o código realmente precisa ganhar desempenho. Para a realização dessa etapa, será utilizado uma ferramenta denominada GNU profiler (gprof) que faz parte do conjunto GNU Binary Utilities (binutils), que tem como principal funcionalidade analisar e capturar os tempos durante a execução do código, gerando relátorios de desempenho. 

Para executar o processo de profilling basta inserir o argumento _-pg_ na compilação do nosso código sequencial, executá-lo normalmente para gerar o arquivo binário do relatório e, logo depois, exibi-lo de forma legível através do comando associados ao gprof, ilustra-se a seguir: 

In [ ]:
!gcc bruteForce.c -o bruteForce -std=c99 -O3 -pg

In [ ]:
!./bruteForce senhatez

In [ ]:
!gprof -b bruteForce gmon.out

<br>&emsp;Após a geração dos relatórios, podemos concluir estratégias de paralelismo para definir os próximos passos do processo de otimização. Pode-se observar que as funções _main_ e _my_pow_ não tem impacto significativo no tempo de execução do algoritmo, em contrapartida, a função _bruteforce_ é responsável por quase 100% do mesmo. Assim sendo, fica fácil de identificar os pontos de gargalo, partindo de um pressuposto que em um código, geralmente, o que mais consome tempo de execução são os laços e temos apenas três deles na função _bruteforce_.
<br> 
&emsp;Os dois primeiros laços são baseados no tamanho da string, enquanto o terceiro possue o maior custo computacional, pois nele será realizada a busca comparativa da chave.
<br>
&emsp;Para um viés de validação, posicionamos capturas de tempo antes e depois de cada laço, imprimindo no terminal o tempo de execução de cada um. Como esperado, os dois primeiros laços representam tempos insiginificantes, enquanto o laço de contagem consome a maior parte do tempo de execução do código. Com o estudo da execução do código realizado, estamos prontos para começar o processo de divisão do domínio e inserção de técnicas de paralelização ao código.

---
## Aplicações Paralelas

### Multicore (OPENMP)

In [ ]:
%%writefile bruteForce-omp.c

/**
TODO
*/

In [ ]:
!gcc bruteForce-omp.c -o bruteForce-omp -fopenmp -std=c99 -O3

In [ ]:
!OMP_NUM_THREADS=16 bruteforce-omp senhatez

### Multiprocessor (MPI)

In [ ]:
%%writefile bruteForce-mpi.c

/**
TODO
*/

In [ ]:
!mpicc bruteForce-mpi.c -o bruteForce-mpi -fopenmp -std=c99 -O3

In [ ]:
!mpirun -np 4 ./bruteForce-mpi senhatez

### Multiprocessor + Multicore (MPI + OpenMP)

In [ ]:
%%writefile bruteForce-mpi+omp.c

/**
TODO
*/

In [ ]:
!mpicc bruteForce-mpi+omp.c -o bruteForce-mpi+omp -fopenmp -std=c99 -O3

In [ ]:
!OMP_NUM_THREADS=4 mpirun -np 4 ./bruteForce-mpi+omp senhatez

### GPU (CUDA)

In [ ]:
%%writefile bruteForce-cuda.cu

/**
TODO
*/

In [ ]:
!nvcc bruteForce-cuda.cu -o bruteForce-cuda -std=c99 -O3

In [ ]:
!./bruteForce senhatez

## Análise Experimental

### I) Validação com Cadeia de zs

#### Parâmetros Ótimos de Execução

1. OpenMP = **A** Threads
2. MPI = **B** nós + **C** Processos
3. MPI + OpenMP = **D** nós + **E** Processos + **F** Threads
4. CUDA = G1D B1DT1D (**G** * 32, 1024)

### Tempos de execução em segundos das aplicações

|  Senha (Entradas)    | Sequencial | OpenMP | MPI  | Híbrido | CUDA
| ---------------------| ---------- | ------ | ---  | ------- | ----
| (7z)  zzzzzzz        |            |        |      |         |  
| (8z)  zzzzzzzz       |            |        |      |         | 
| (9z)  zzzzzzzzz      |            |        |      |         | 
| (10z) zzzzzzzzzz     |            |        |      |         | 

### Speedups

|  Senha    |  OpenMP    | MPI     | Híbrido       | CUDA
| --------- |  ------    | ------  | -------       | ----
| 7z        |            |         |               |  
| 8z        |            |         |               |   
| 9z        |            |         |               |  
| 10z       |            |         |               |  

### II) Análise de Desempenho - Senha Específica: -Hacka01

#### Parâmetros Ótimos de Execução

1. OpenMP = A Threads
2. MPI = B nós + C Processos
3. MPI + OpenMP = D nós + E Processos + F Threads
4. CUDA = G1D B1DT1D (G * 32, 1024)

### Tempo de execução em segundos das aplicações

|  Senha (Entradas)    | Sequencial | OpenMP | MPI  | Híbrido | CUDA
| ---------------------| ---------- | ------ | ---  | ------- | ----
| -Hacka01              |            |        |      |         |  


### Speedup

|  Senha    |  OpenMP    | MPI     | Híbrido       | CUDA
| --------- |  ------    | ------  | -------       | ----
| -Hacka01   |            |         |               |  


## Conclusões

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

## Referências Biliográficas

* G. Coulouris, J. Dollimore, T. Kindberg, G.Blair. Distributed Systems: Concepts and Design, Fifth Edition, Pearson, 2011.

* S.Tanenbaum, M. Steen, Distributed Systems: Principles and Paradigms, Second Edition, Pearson, 2006.

* David A. Patterson and John L. Hennessy. Computer Organization and Design: The Hardware/Software Interface. Morgan Kaufmann, 5th Edition, 2013.

* An Introduction to Parallel Programming by Peter S. Pacheco. Morgan Kauffman.

* W. C. Barbosa, An introduction to distributed algorithms, MIT Press, 1997. N. Lynch, Distributed Algorithms, Mit Press, 1996 e Introduction to Distributed Algorithms, Gerard Tel, Cabribridge U. Press, 1994.